In [1]:
import ale_py
import gym
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import tqdm
import imageio
import random
import tensorflow as tf
import os

In [2]:
env = gym.make("PongDeterministic-v4")

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [3]:
def preprocess_frame(image):
    image = image[35:195:2, ::2, 0]
    image = torch.from_numpy(image).float()
    image[image == 144] = 0
    image[image == 109] = 0
    image[image != 0] = 1
    return image.flatten()


In [4]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(80*80, 200),
            nn.ReLU(),
            nn.Linear(200, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

In [5]:
def play_episode(policy,env):
    UP = 2
    DOWN = 3
    observation, _ = env.reset()    
    prev_state = None
    state = preprocess_frame(observation)
    probs = []
    rewards = []
    while True:
        if prev_state is not None:
            change = state - prev_state
        else:
            change = state
        prob = policy(change).squeeze(0)
        action = UP if random.random() < prob.item() else DOWN
        observation, reward, terminated, truncated, _ = env.step(action)
        prev_state = state
        state = preprocess_frame(observation)
        probs.append(prob if action == UP else 1 - prob)
        rewards.append(reward)
        if terminated or truncated:
            break
    return torch.stack(probs), torch.tensor(rewards)

In [6]:
def calculate_returns(rewards,gamma):
    discounted_r = []
    running_add = 0
    for reward in reversed(rewards):
        if reward != 0:
            running_add = 0
        running_add = running_add * gamma + reward
        discounted_r.append(running_add)
    return torch.tensor(discounted_r[::-1])

In [7]:
def train_policy(
    policy,
    env,
    gamma,
    epochs=1000,
    batch_size=4,
    n_save_epochs=50,
    checkpoint_path="checkpoint.pth",
    load_checkpoint=True,
):
    writer = tf.summary.create_file_writer("logs")
    writer.set_as_default()

    if load_checkpoint:
        if os.path.exists(checkpoint_path):
            policy.load_state_dict(torch.load(checkpoint_path))
        else:
            print(f"Checkpoint {checkpoint_path} not found")
            
    optimizer = torch.optim.Adam(policy.parameters(), lr=7e-4)
    policy.train()
    for epoch in tqdm.tqdm(range(epochs)):
        mean_batch_loss = 0
        mean_batch_reward = 0
        for _ in range(batch_size):
            probs, rewards = play_episode(policy, env)
            returns = calculate_returns(rewards, gamma)
            returns = (returns - returns.mean()) / returns.std()
            loss = -(returns * torch.log(probs)).sum()
            mean_batch_loss += loss / batch_size
            mean_batch_reward += rewards.sum() / batch_size
        optimizer.zero_grad()
        mean_batch_loss.backward()
        optimizer.step()

        print(
            f"Epoch: {epoch}, mean loss: {mean_batch_loss:.2f}, "
            f"mean reward: {mean_batch_reward:.2f}"
        )
        tf.summary.scalar('mean loss', mean_batch_loss.detach().item(), step=epoch)
        tf.summary.scalar('mean reward', mean_batch_reward.detach().item(), step=epoch)

        if epoch % n_save_epochs == 0:
            torch.save(policy.state_dict(), checkpoint_path)
            print(f"Checkpoint for {epoch} epoch saved to {checkpoint_path}")

In [8]:
policy = PolicyNetwork()
train_policy(
    policy,
    env,
    gamma=0.99,
    epochs=2000,
    batch_size=4,
    n_save_epochs=50,
    checkpoint_path="checkpoint.pth",
    load_checkpoint=True,
)

Checkpoint checkpoint.pth not found


  0%|          | 0/2000 [00:00<?, ?it/s]/Users/nikita/anaconda3/envs/myenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
  0%|          | 1/2000 [00:05<3:19:32,  5.99s/it]

Epoch: 0, mean loss: 0.12, mean reward: -20.50


  0%|          | 2/2000 [00:12<3:24:33,  6.14s/it]

Epoch: 1, mean loss: -0.02, mean reward: -21.00


...

Epoch: 801, mean loss: -49.99, mean reward: 20.50


 40%|████      | 803/2000 [3:21:40<5:07:48, 15.43s/it]

Epoch: 802, mean loss: -57.57, mean reward: 20.25


 40%|████      | 803/2000 [3:21:43<5:00:42, 15.07s/it]


KeyboardInterrupt: 

In [9]:
torch.save(policy.state_dict(), "pong_policy.pth")

In [10]:
def create_gameplay_gif(policy,env,file_name="pong_gameplay.gif"):
    UP = 2
    DOWN = 3
    policy.eval()
    observation, _ = env.reset()    
    frames = [observation]
    prev_state = None
    state = preprocess_frame(observation)
    while True:
        if prev_state is not None:
            change = state - prev_state
        else:
            change = state
        prob = policy(change).squeeze(0)
        action = UP if random.random() < prob.item() else DOWN
        observation, reward, terminated, truncated, _ = env.step(action)
        prev_state = state
        state = preprocess_frame(observation)
        frames.append(observation)
        if terminated or truncated:
            break
    imageio.mimsave(file_name, frames, fps=30)

In [11]:
create_gameplay_gif(policy,env,"pong_gameplay1.gif")

In [9]:
env.close()